In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pyupbit



In [40]:
'''
목표 : 당일 시가가 전날 시가보다 높은지 낮은지 예측
구성 피쳐
- 전날 시가, 종가, 고가, 종가, 거래량
- 전날 유의종목 지정 여부
- 전날 주의종목 지정 여부 (
    - PRICE_FLUCTUATIONS: 가격 급등락 경보 발령 여부
    - TRADING_VOLUME_SOARING: 거래량 급등 경보 발령 여부
    - DEPOSIT_AMOUNT_SOARING: 입금량 급등 경보 발령 여부
    - GLOBAL_PRICE_DIFFERENCES: 가격 차이 경보 발령 여부
    - CONCENTRATION_OF_SMALL_ACCOUNTS: 소수 계정 집중 경보 발령 여부
)
'''

import requests
import pandas as pd
import json

url = "https://api.upbit.com/v1/candles/minutes/1?market=KRW-BTC&to=2023-01-01T00:00:00&count=200"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

print(response.text)

[{"market":"KRW-BTC","candle_date_time_utc":"2022-12-31T23:59:00","candle_date_time_kst":"2023-01-01T08:59:00","opening_price":21069000.00000000,"high_price":21079000.00000000,"low_price":21065000.00000000,"trade_price":21079000.00000000,"timestamp":1672531199722,"candle_acc_trade_price":78262323.62502000,"candle_acc_trade_volume":3.71431684,"unit":1},{"market":"KRW-BTC","candle_date_time_utc":"2022-12-31T23:58:00","candle_date_time_kst":"2023-01-01T08:58:00","opening_price":21080000.00000000,"high_price":21080000.00000000,"low_price":21066000.00000000,"trade_price":21069000.00000000,"timestamp":1672531135644,"candle_acc_trade_price":25685160.85060000,"candle_acc_trade_volume":1.21882754,"unit":1},{"market":"KRW-BTC","candle_date_time_utc":"2022-12-31T23:57:00","candle_date_time_kst":"2023-01-01T08:57:00","opening_price":21080000.00000000,"high_price":21080000.00000000,"low_price":21071000.00000000,"trade_price":21071000.00000000,"timestamp":1672531073105,"candle_acc_trade_price":55352

In [41]:
# JSON 문자열을 딕셔너리로 변환
data_list = json.loads(response.text)

# 판다스 데이터프레임 생성
df = pd.DataFrame(data_list)

# candle_date_time_kst 컬럼을 datetime 형태로 변환
df['candle_date_time_kst'] = pd.to_datetime(df['candle_date_time_kst'])

# 년, 월, 일, 시간, 분 컬럼 생성
df['year'] = df['candle_date_time_kst'].dt.year
df['month'] = df['candle_date_time_kst'].dt.month
df['day'] = df['candle_date_time_kst'].dt.day
df['hour'] = df['candle_date_time_kst'].dt.hour
df['minute'] = df['candle_date_time_kst'].dt.minute

In [42]:
len(data_list)

200

In [36]:
df['candle_date_time_utc'].iloc[-1]

'2022-12-31T20:40:00'

# 데이터 수집

In [131]:
def btc_collection(market, date, count=200):
    check_cnt = 0
    headers = {"accept": "application/json"}
    # df_all = pd.DataFrame()
    while True:
    # API 호출
        response = requests.get(
                url = f"https://api.upbit.com/v1/candles/minutes/1?market={market}&to={date}&count={count}"
            , headers=headers
            )
        data = json.loads(response.text)

        # 더 이상 데이터가 없으면 반복문 종료
        if not data:
            print('no data')
            break
        
        # 데이터를 데이터프레임으로 변환
        if check_cnt == 0:
        # 새로운 데이터를 기존 데이터프레임에 추가
            df = pd.DataFrame(data)
            df_all = pd.DataFrame(data)
        
        else:
            df = pd.DataFrame(data)
            df_all = pd.concat([df_all, df])
        
        if len(data) != 200:
            print('last')
            break
        
        # 다음 데이터를 가져오기 위한 to_date 갱신
        date = df_all['candle_date_time_kst'].iloc[-1]
        
        # print(
        #     df['candle_date_time_kst'].iloc[-1], ' ~ ', df['candle_date_time_kst'].iloc[0]
        # )
        print(date)
        
        check_cnt += 1
        
    return df_all

In [135]:
data = btc_collection('KRW-BTC', '2018-09-01T00:00:00')

2018-09-01T05:38:00
2018-09-01T11:18:00
2018-09-01T16:58:00
2018-09-01T22:38:00
2018-09-02T03:25:00
2018-09-02T09:05:00
2018-09-02T14:45:00
2018-09-02T20:25:00
2018-09-03T01:53:00
2018-09-03T07:31:00
2018-09-03T13:11:00
2018-09-03T18:51:00
2018-09-04T00:27:00
2018-09-04T06:04:00
2018-09-04T11:44:00
2018-09-04T17:24:00
2018-09-04T23:04:00
2018-09-05T04:42:00
2018-09-05T10:22:00
2018-09-05T16:02:00
2018-09-05T21:42:00
2018-09-06T03:21:00
2018-09-06T09:01:00
2018-09-06T14:41:00
2018-09-06T20:21:00
2018-09-07T01:52:00
2018-09-07T07:32:00
2018-09-07T13:12:00
2018-09-07T18:52:00
2018-09-08T00:26:00
2018-09-08T06:02:00
2018-09-08T11:39:00
2018-09-08T17:17:00
2018-09-08T22:52:00
2018-09-09T04:32:00
2018-09-09T10:12:00
2018-09-09T15:49:00
2018-09-09T21:28:00
2018-09-10T02:38:00
2018-09-10T08:18:00
2018-09-10T13:58:00
2018-09-10T19:38:00
2018-09-11T01:15:00
2018-09-11T06:52:00
2018-09-11T12:32:00
2018-09-11T18:12:00
2018-09-11T23:51:00
2018-09-12T05:30:00
2018-09-12T11:10:00
2018-09-12T16:50:00


In [92]:
data

""


In [122]:
check_cnt = 0
headers = {"accept": "application/json"}
date = '2018-09-01T00:00:00'
# df_all = pd.DataFrame()
while True:
# API 호출
    response = requests.get(
            url = f"https://api.upbit.com/v1/candles/minutes/1?market=KRW-BTC&to={date}&count=200"
        , headers=headers
        )
    print(date)
    data = json.loads(response.text)

    # 더 이상 데이터가 없으면 반복문 종료
    if not data:
        print('no data')
        break
    
    # 데이터를 데이터프레임으로 변환
    if check_cnt == 0:
    # 새로운 데이터를 기존 데이터프레임에 추가
        df = pd.DataFrame(data)
        df_all = pd.DataFrame(data)
    
    else:
        df = pd.DataFrame(data)
        df_all = pd.concat([df_all, df])
    
    if len(data) != 200:
        print('last')
        break
    
    # 다음 데이터를 가져오기 위한 to_date 갱신
    date = df_all['candle_date_time_kst'].iloc[-1]
    
    # print(
    #     df['candle_date_time_kst'].iloc[-1], ' ~ ', df['candle_date_time_kst'].iloc[0]
    # )
    check_cnt += 1
    if check_cnt == 3:
        break

2018-09-01T00:00:00
2018-09-01T05:38:00
2018-09-01T11:18:00


In [125]:
df['candle_date_time_kst'].iloc[0]

'2018-09-01T20:17:00'

In [127]:
df_all.iloc[190:220]

,market,candle_date_time_utc,candle_date_time_kst,opening_price,high_price,low_price,trade_price,timestamp,candle_acc_trade_price,candle_acc_trade_volume,unit
190,KRW-BTC,2018-08-31T20:47:00,2018-09-01T05:47:00,7885000.0,7885000.0,7870000.0,7878000.0,1535748476722,7.568211e+06,0.960540,1
191,KRW-BTC,2018-08-31T20:46:00,2018-09-01T05:46:00,7886000.0,7886000.0,7885000.0,7885000.0,1535748419314,1.807173e+06,0.229182,1
192,KRW-BTC,2018-08-31T20:45:00,2018-09-01T05:45:00,7880000.0,7885000.0,7880000.0,7880000.0,1535748353201,4.995502e+06,0.633704,1
193,KRW-BTC,2018-08-31T20:44:00,2018-09-01T05:44:00,7879000.0,7880000.0,7879000.0,7880000.0,1535748271041,1.009904e+07,1.281735,1
194,KRW-BTC,2018-08-31T20:43:00,2018-09-01T05:43:00,7880000.0,7880000.0,7879000.0,7879000.0,1535748237417,3.722996e+06,0.472515,1
195,KRW-BTC,2018-08-31T20:42:00,2018-09-01T05:42:00,7886000.0,7886000.0,7879000.0,7879000.0,1535748175797,4.808801e+06,0.610284,1
196,KRW-BTC,2018-08-31T20:41:00,2018-09-01T05:41:00,7886000.0,7886000.0,7879000.0,7886000.0,1535748099844,9.018639e+06,1.143860,1
197,KRW-BTC,2018-08-31T20:40:00,2018-09-01T05:40:00,7889000.0,7889000.0,7878000.0,7886000.0,1535748059914,1.655344e+07,2.100425,1
198,KRW-BTC,2018-08-31T20:39:00,2018-09-01T05:39:00,7877000.0,7883000.0,7877000.0,7883000.0,1535747981202,1.692360e+07,2.147994,1
199,KRW-BTC,2018-08-31T20:38:00,2018-09-01T05:38:00,7878000.0,7878000.0,7871000.0,7878000.0,1535747921027,1.711992e+07,2.174797,1
